In [2]:
import pandas as pd 
import joblib

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

데이터셋

In [3]:
df_train = pd.read_csv('../static/data/iris_train.csv')
df_train.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.3,1.7,0.5,0
1,6.1,3.0,4.9,1.8,2
2,5.1,3.4,1.5,0.2,0
3,5.1,3.5,1.4,0.3,0
4,5.2,3.4,1.4,0.2,0


정규화 시키고,

In [4]:
y_train = df_train.target.values
scaler = MinMaxScaler()
X_train = scaler.fit_transform(df_train.drop(columns='target', axis=1))
X_train.shape, y_train.shape

((112, 4), (112,))

In [5]:
#테스트 df_test
df_test = pd.read_csv('../static/data/iris_test.csv')
df_test.head(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.8,3.0,1.4,0.3,0
1,4.6,3.6,1.0,0.2,0
2,4.8,3.0,1.4,0.1,0
3,4.7,3.2,1.6,0.2,0
4,4.9,2.4,3.3,1.0,1
5,5.7,2.9,4.2,1.3,1
6,6.3,2.8,5.1,1.5,2
7,6.4,2.7,5.3,1.9,2
8,6.0,3.0,4.8,1.8,2
9,6.0,2.2,5.0,1.5,2


In [6]:
y_test = df_test.target.values
X_test = scaler.fit_transform(df_test.drop(columns='target', axis=1))
X_test.shape, y_test.shape
#X_test = df_test.iloc[:, :-1].values
#y_test = df_test.iloc[:, -1].values

((38, 4), (38,))

In [7]:
joblib.dump(scaler, '../static/model/iris_scaler.pkl')

['../static/model/iris_scaler.pkl']

1. Logistic Regression

In [8]:
lr_clf = LogisticRegression()
lr_clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [9]:
params = {
    'C': [0.1, 1, 5, 10]  
}

In [10]:
grid_cv = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)


최고 평균 정확도: 0.9739
최적 파라미터: {'C': 5}


In [11]:
best_lr = grid_cv.best_estimator_
pred = best_lr.predict(X_test)
accuracy_score(y_test, pred)

0.9210526315789473

In [12]:
joblib.dump(best_lr, '../static/model/iris_lr.pkl')

['../static/model/iris_lr.pkl']

2.SVM

In [13]:
svc = SVC()
svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [14]:
params = {
    'C': [0.1, 1, 5, 7, 10]  
}

In [15]:
grid_cv = GridSearchCV(svc, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.9826
최적 파라미터: {'C': 5}


In [16]:
best_sv = grid_cv.best_estimator_
pred = best_sv.predict(X_test)
accuracy_score(y_test, pred)

0.9210526315789473

In [17]:
joblib.dump(best_sv, '../static/model/iris_sv.pkl')

['../static/model/iris_sv.pkl']

3.Random Forest

In [18]:
rf_clf = RandomForestClassifier()
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [19]:
params = {
    'n_estimators': [10, 50, 100],
    'max_depth': [4, 6, 8, 10],
    'min_samples_split': [2, 3, 4]
}

In [20]:
grid_cv = GridSearchCV(rf_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.9913
최적 파라미터: {'max_depth': 8, 'min_samples_split': 2, 'n_estimators': 100}


In [21]:
best_rf = grid_cv.best_estimator_
pred = best_rf.predict(X_test)
accuracy_score(y_test, pred)

0.9210526315789473

In [22]:
joblib.dump(best_rf, '../static/model/iris_rf.pkl')

['../static/model/iris_rf.pkl']

Test

In [23]:
index = 4
test_data = (df_test.iloc[index, :-1].values)
label = df_test.iloc[index, -1]

test_data,label

(array([4.9, 2.4, 3.3, 1. ]), 1)

In [24]:
test_data = test_data.reshape(1,-1) #1차원 데이터는 predict를 할수 없다.그래서 reshape 함
test_data

array([[4.9, 2.4, 3.3, 1. ]])

In [25]:
new_scaler = joblib.load('../static/model/iris_scaler.pkl')

In [26]:
test_scaled = new_scaler.transform(test_data)
test_scaled

array([[0.17857143, 0.125     , 0.46      , 0.40909091]])

In [27]:
pred_lr = best_lr.predict(test_scaled)
pred_sv = best_sv.predict(test_scaled)
pred_rf = best_rf.predict(test_scaled)

In [28]:
label, pred_lr[0], pred_sv[0], pred_rf[0]

(1, 1, 1, 1)

feature_name, feature_value 를 갖는 딕셔너리 만들기

In [29]:
df_test.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.8,3.0,1.4,0.3,0
1,4.6,3.6,1.0,0.2,0
2,4.8,3.0,1.4,0.1,0
3,4.7,3.2,1.6,0.2,0
4,4.9,2.4,3.3,1.0,1


In [30]:
a = dict(zip(df_test.columns[:], df_test.iloc[index, :]))
a

{'sepal length (cm)': 4.9,
 'sepal width (cm)': 2.4,
 'petal length (cm)': 3.3,
 'petal width (cm)': 1.0,
 'target': 1.0}